In [ ]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [ ]:
from pathlib import Path
from src.utils.config_loader import load_config
from src.utils.seed import seed_everything

base_dir = Path(os.getcwd()).parent

config = load_config(base_dir / 'model_params.yaml')

seed_everything(config['general']['seed'])

In [ ]:
import json

def format_examples(example_subset): 
    # Formats the examples into a string for later prompt
    formatted = []
    for i, ex in enumerate(example_subset):
        entity_lines = "\n".join([f"{e['word']} {e['label']}" for e in ex["entities"]])
        formatted.append(f"Example {i+1}:\nSentence: \"{ex['sentence']}\"\nEntities:\n{entity_lines}\n")
    
    return "\n".join(formatted)

with open(base_dir / 'llm_stuff/prompts/examples.json', 'r') as f:
    example_bank = json.load(f)

ids = [3, 13, 9, 1, 25] 

examples = [next(ex for ex in example_bank if ex["id"] == id) for id in ids]

formatted_examples = format_examples(examples)

print(formatted_examples)

In [ ]:
import torch
import transformers
from collections import defaultdict
from src.utils.label_mapping_regplans import id_to_label, label_to_id

class LlamaForNER:
    def __init__(self, model_path: str):
        self.model_id = model_path
        self.pipeline = transformers.pipeline(
            'text-generation',
            model=self.model_id,
            model_kwargs={'torch_dtype': torch.float16} # FP16 for faster inference
        )
        tokenizer = self.pipeline.tokenizer
        tokenizer.pad_token_id = tokenizer.eos_token_id 

        self.terminators = tokenizer.eos_token_id
  
    def format_prompt(self, sentence):
        prompt = f"""
        You are an expert in Natural Language Processing. Your task is to identify Named Entities (NER) in a given text.
        The possible Named Entities are exclusively 'B-FELT' and 'I-FELT'. The entities are defined as follows:

        - B-FELT: The beginning of a field zone name.
        - I-FELT: The continuation of a field zone name.   
                        
        ### Examples:
            
        {formatted_examples}
            
        Your task is to identify the Named Entities in the following sentence: '{sentence}'
        """
        return prompt

    def predict_ner(self, sentence, tokens, max_tokens=50, temperature=0.0, top_p=0.9):

        prompt = self.format_prompt(sentence) 

        outputs = self.pipeline(
            prompt,
            max_new_tokens=max_tokens,
            eos_token_id=self.terminators,
            pad_token_id=self.terminators,
            do_sample=False, # True could be useful
            temperature=temperature, # lower = more deterministic
            top_p=top_p 
        )
        
        generated_text = outputs[0]['generated_text'][len(prompt):].strip()
        pred_labels = self.extract_entities(generated_text, tokens)
        
        return {
            'tokens': tokens,
            'pred_labels': pred_labels,
            'generated_text': generated_text
        }

    def extract_entities(self, output_text, tokens):

        entities = defaultdict(list) # Word-label pairs

        for line in output_text.splitlines():
            parts = line.strip().split()
            if len(parts) == 2:
                word, label = parts[0], parts[1]
                entities[word].append(label)

        pred_labels = []
        word_counts = defaultdict(int)  # Track occurrences of each word
    
        for token in tokens:
            if token in entities and word_counts[token] < len(entities[token]):
                pred_labels.append(entities[token][word_counts[token]])  # Get the label in order
                word_counts[token] += 1  # Increment occurrence counter
            else:
                pred_labels.append("O")  # Default to "O" if missing
        
        return pred_labels 

In [ ]:
from src.data.preprocessing import create_df
from llm_stuff.evaluation import evaluate 
from tqdm import tqdm
import json

val_df = create_df(base_dir / 'data/my_data/regplans-dev.conllu')

model_path = 'meta-llama/Meta-Llama-3-8B-Instruct' 
ner_model = LlamaForNER(model_path)

all_pred_ids = []
all_true_ids = []

all_results = []

for idx, row in tqdm(val_df.iterrows(), total=len(val_df)):
    sentence = row['full_text']
    tokens = row['words']
    true_labels = row['labels']  

    output = ner_model.predict_ner(sentence, tokens)
    pred_labels = output['pred_labels']

    pred_ids = []
    for label in pred_labels:
        if label in label_to_id:  
            pred_ids.append(label_to_id[label])
        else:
            print(f"Warning: Unexpected label '{label}' found. Assigning default label 'O'.")
            pred_ids.append(label_to_id.get("O", -1))

    true_ids = [label_to_id[label] for label in true_labels]

    all_pred_ids.extend(pred_ids)
    all_true_ids.extend(true_ids)

    all_results.append({
        'sentence': sentence,
        'tokens': tokens,
        'true_labels': true_labels,
        'predicted_labels': pred_labels,
        'generated_text': output['generated_text']
    })

    #print(f"Sentence: {sentence}")
    #print(f"Predicted Labels: {pred_labels}")
    #print(f"True Labels: {true_labels}")

# Calculate evaluation metrics across all tokens in the dev set.
metrics = evaluate(all_true_ids, all_pred_ids)

print("Evaluation Metrics on Dev Set:")
print(metrics)

final_output = {
    'prompt': ner_model.format_prompt(''),
    'evaluation_metrics': metrics,
    'results': all_results
}

with open(base_dir / f'llm_stuff/results/Meta-Llama-3-8B-Instruct_5ex.json', 'w', encoding='utf-8') as f:
    json.dump(final_output, f, indent=4, ensure_ascii=False)